In [1]:
from hyperopt import fmin, hp, tpe, Trials
import numpy as np

import os
import sys

BASEDIR = os.path.dirname(os.getcwd())
sys.path.append(BASEDIR)

from copy import deepcopy
from functools import partial
from dgl.dataloading import GraphDataLoader
from src.model.attentivefp_postnet import attentivefpPostNet
from src.utils.mol.attfp_graph import MoleculeDataset, collate_fn
from src.config.attentivefp_postnet import attentivefpPostNetArgs
from src.pipeline.ensemble import training_ensemble_models
from src.utils.basic.logger import Writer, NpEncoder
from src.utils.model.metrics import accuracy, roc_auc_score, prc_auc, F1, MCC, expected_calibration_error, OverconfidentFalseRate, OverconfidentFalseNegatives, Brier
import torch
import pandas as pd
import json

# Initialize

In [2]:
torch.set_num_threads(4)

target_name = 'CardioTox'
gpu_num     = 0

# Bayesian optimization searching space

SPACE = {
    'hidden_size'       : hp.quniform('hidden_size', low=300, high=600, q=100),
    'radius'            : hp.quniform('radius', low=2, high=6, q=1),
    'T'                 : hp.quniform('T', low=1, high=5, q=1),
    'p_dropout'         : hp.quniform('dropout', low=0.0, high=0.5, q=0.05),
    'init_lr'           : hp.loguniform('init_lr', low=np.log(1e-4), high=np.log(1e-2)),
    'ffn_num_layers'    : hp.quniform('ffn_num_layers', low=2, high=4, q=1),
    'density_type'      : hp.choice('density_type', ['iaf_flow']),
    'n_density'         : hp.quniform('n_density', low=2, high=10, q=1),
    'latent_dim'        : hp.quniform('latent_dim', low=2, high=10, q=1)
}

INT_KEYS = ['hidden_size', 'radius', 'T', 'ffn_num_layers', 'latent_dim', 'n_density']
SAVEDIR = os.path.join(BASEDIR, "results", target_name, 'AttFpPost')
DATADIR = os.path.join(BASEDIR, "data", target_name)

# Training function

In [4]:
n = 0
logger = Writer(os.path.join(SAVEDIR, "history.log"))
def func(hyperparams):
    logger(" ")
    logger(" ")

    global n
    n = n+1
    logger(f"ROUND {n}")

    BASESAVEDIR = os.path.join(SAVEDIR, f"ROUND_{n}")
    logger_writer = Writer(os.path.join(BASESAVEDIR, "config.txt"))
    for k in hyperparams:
        logger_writer(f"{k}\t{hyperparams[k]}")

    for key in INT_KEYS:
        hyperparams[key] = int(hyperparams[key])

    config = attentivefpPostNetArgs().parse_args([], known_only=True)
    hyper_args = deepcopy(config)

    for key, value in hyperparams.items():
        setattr(hyper_args, key, value)

    setattr(hyper_args, "dataset_type", "classification")
    setattr(hyper_args, 'metric', "roc-auc")
    setattr(hyper_args, "extra_metrics", ["MCC", "prc-auc", "accuracy", "F1"])
    setattr(hyper_args, "ffn_hidden_size", hyper_args.hidden_size)
    setattr(hyper_args, "early_stopping_num", 30)
    setattr(hyper_args, "gpu", gpu_num)
    setattr(hyper_args, "log_frequency", 100)
    setattr(hyper_args, "batch_size", 128)
    setattr(hyper_args, "at_least_epoch", 0)
    print(hyper_args)

    for i in [1, 2, 3, 4, 5]:

        train_dataset = MoleculeDataset(os.path.join(DATADIR, f"{target_name}_train.csv"))
        valid_dataset = MoleculeDataset(os.path.join(DATADIR, f"{target_name}_valid.csv"))
        test_dataset  = MoleculeDataset(os.path.join(DATADIR, f"{target_name}_test.csv"))

        train_targets = []
        for _, t in train_dataset:
            train_targets.append(t)

        N = torch.tensor([len(train_targets) - np.sum(train_targets).astype(np.int64),
                          np.sum(train_targets).astype(np.int64)], dtype=torch.float64)

        setattr(hyper_args, "N", N)
        
        train_dataloader = GraphDataLoader(dataset=train_dataset, collate_fn=collate_fn, batch_size=128, drop_last=False, shuffle=True)
        train_dataloader.smiles = [[s] for s in train_dataset.smiles_list]
        valid_dataloader = GraphDataLoader(dataset=valid_dataset, collate_fn=collate_fn, batch_size=128, drop_last=False, shuffle=False)
        valid_dataloader.smiles = [[s] for s in valid_dataset.smiles_list]
        test_dataloader  = GraphDataLoader(dataset=test_dataset,  collate_fn=collate_fn, batch_size=128, drop_last=False, shuffle=False)
        test_dataloader.smiles  = [[s] for s in test_dataset.smiles_list]

        training_ensemble_models(os.path.join(BASESAVEDIR, f"fold_{i}"),
                                 attentivefpPostNet,
                                 hyper_args,
                                 train_dataloader,
                                 valid_dataloader=valid_dataloader,
                                 test_dataloader=test_dataloader,
                                 ensemble_num=1)
        
        test_prediction = []
        
        valid_ROC = []
        valid_PRC = []
        valid_ACC = []
        valid_MCC = []
        valid_F1  = []
        valid_ECE = []
        valid_OFR = []
        valid_OFN = []
        valid_Brier = []
        
        test_ROC  = []
        test_PRC  = []
        test_ACC  = []
        test_MCC  = []
        test_F1   = []
        test_ECE  = []
        test_OFR = []
        test_OFN = []
        test_Brier = []

    for i in [1, 2, 3, 4, 5]:
    
        temp_dir = os.path.join(BASESAVEDIR, f"fold_{i}", "model_0")
        
        temp_valid_prediction = pd.read_csv(os.path.join(temp_dir, "valid_prediction.csv"))["property_pred"].to_numpy()
        temp_valid_label = pd.read_csv(os.path.join(temp_dir, "valid_prediction.csv"))["property_label"].to_numpy()
        
        temp_test_prediction = pd.read_csv(os.path.join(temp_dir, "test_prediction.csv"))["property_pred"].to_numpy()
        test_label = pd.read_csv(os.path.join(temp_dir, "test_prediction.csv"))["property_label"].to_numpy()
        test_prediction.append(temp_test_prediction) # for ensemble calculation
        
        valid_ROC.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["roc-auc"].iloc[0])
        valid_PRC.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["prc-auc"].iloc[0])
        valid_ACC.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["accuracy"].iloc[0])
        valid_MCC.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["MCC"].iloc[0])
        valid_F1.append(pd.read_csv(os.path.join(temp_dir, "valid_prediction_performance.csv"))["F1"].iloc[0])
        valid_ECE.append(expected_calibration_error(temp_valid_label, temp_valid_prediction, bins=10))
        valid_OFR.append(OverconfidentFalseRate(temp_valid_prediction, temp_valid_label))
        valid_OFN.append(OverconfidentFalseNegatives(temp_valid_prediction, temp_valid_label))
        valid_Brier.append(Brier(temp_valid_label,temp_valid_prediction))
        
        test_ROC.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["roc-auc"].iloc[0])
        test_PRC.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["prc-auc"].iloc[0])
        test_ACC.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["accuracy"].iloc[0])
        test_MCC.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["MCC"].iloc[0])
        test_F1.append(pd.read_csv(os.path.join(temp_dir, "test_prediction_performance.csv"))["F1"].iloc[0])
        test_ECE.append(expected_calibration_error(test_label, temp_test_prediction, bins=10))
        test_OFR.append(OverconfidentFalseRate(temp_test_prediction, test_label))
        test_OFN.append(OverconfidentFalseNegatives(temp_test_prediction, test_label))
        test_Brier.append(Brier(test_label,temp_test_prediction))
        

    logger(f'ROUND {n} Valid ROC-AUC {np.mean(valid_ROC)} +/- {np.std(valid_ROC)}')
    logger(f'ROUND {n} Valid PRC-AUC {np.mean(valid_PRC)} +/- {np.std(valid_PRC)}')
    logger(f'ROUND {n} Valid ACC     {np.mean(valid_ACC)} +/- {np.std(valid_ACC)}')
    logger(f'ROUND {n} Valid MCC     {np.mean(valid_MCC)} +/- {np.std(valid_MCC)}')
    logger(f'ROUND {n} Valid F1      {np.mean(valid_F1)} +/- {np.std(valid_F1)}')
    logger(f'ROUND {n} Valid ECE     {np.mean(valid_ECE)} +/- {np.std(valid_ECE)}')
    logger(f'ROUND {n} Valid OFR     {np.mean(valid_OFR)} +/- {np.std(valid_OFR)}')
    logger(f'ROUND {n} Valid OFN     {np.mean(valid_OFN)} +/- {np.std(valid_OFN)}')
    logger(f'ROUND {n} Valid Brier   {np.mean(valid_Brier)} +/- {np.std(valid_Brier)}')
    logger(' ')
    logger(f'ROUND {n} Test ROC-AUC {np.mean(test_ROC)} +/- {np.std(test_ROC)}')
    logger(f'ROUND {n} Test PRC-AUC {np.mean(test_PRC)} +/- {np.std(test_PRC)}')
    logger(f'ROUND {n} Test ACC     {np.mean(test_ACC)} +/- {np.std(test_ACC)}')
    logger(f'ROUND {n} Test MCC     {np.mean(test_MCC)} +/- {np.std(test_MCC)}')
    logger(f'ROUND {n} Test F1      {np.mean(test_F1)} +/- {np.std(test_F1)}')
    logger(f'ROUND {n} Test ECE     {np.mean(test_ECE)} +/- {np.std(test_ECE)}')
    logger(f'ROUND {n} Test OFR     {np.mean(test_OFR)} +/- {np.std(test_OFR)}')
    logger(f'ROUND {n} Test OFN     {np.mean(test_OFN)} +/- {np.std(test_OFN)}')
    logger(f'ROUND {n} Test Brier   {np.mean(test_Brier)} +/- {np.std(test_Brier)}')
    logger(' ')
    logger(f'ROUND {n} Ensemble Test ROC-AUC {roc_auc_score(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test PRC-AUC {prc_auc(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test ACC {accuracy(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test MCC {MCC(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test F1 {F1(test_label, np.mean(test_prediction, axis=0))}')
    logger(f'ROUND {n} Ensemble Test ECE {expected_calibration_error(test_label, np.mean(test_prediction, axis=0), bins=10)}')
    logger(f'ROUND {n} Ensemble Test OFR {OverconfidentFalseRate(np.mean(test_prediction, axis=0), test_label)}')
    logger(f'ROUND {n} Ensemble Test OFN {OverconfidentFalseNegatives(np.mean(test_prediction, axis=0), test_label)}')
    logger(f'ROUND {n} Ensemble Test Brier {Brier(test_label, np.mean(test_prediction, axis=0))}')
    logger(' ')

    return -np.mean(valid_ROC)



algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(func, SPACE, algo=algo, max_evals=2)  # best is a dictionary
json_str = json.dumps(best, cls=NpEncoder)  # using json to turn a dictionary to a str 
logger(json_str)

  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]07-15 06:41:13	 
07-15 06:41:13	 
07-15 06:41:13	ROUND 1
07-15 06:41:13	T	4.0
07-15 06:41:13	density_type	iaf_flow


07-15 06:41:13	ffn_num_layers	2.0
07-15 06:41:13	hidden_size	400.0
07-15 06:41:13	init_lr	0.002221362095939982
07-15 06:41:13	latent_dim	2.0
07-15 06:41:13	n_density	6.0
07-15 06:41:13	p_dropout	0.05
07-15 06:41:13	radius	2.0
{'N': [100, 100],                                    
 'T': 4,
 'activation': 'ReLU',
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'at_least_epoch': 0,
 'atom_descriptors': None,
 'atom_descriptors_size': None,
 'atom_messages': False,
 'batch_size': 128,
 'bias': False,
 'cuda': True,
 'dataset_type': 'classification',
 'density_type': 'iaf_flow',
 'depth': 3,
 'device': device(type='cuda', index=0),
 'dropout': 0.0,
 'early_stopping_num': 30,
 'empty_cache': False,
 'explicit_h': False,
 'extra_metrics': ['MCC', 'prc-auc', 'accuracy', 'F1'],
 'features_only': False,
 'features_size': None,
 'ffn_hidden_size': 400,
 'ffn_num_layers': 2,
 'fingerprint_dim': 200,
 'gpu': 0,
 'grad_clip': None,
 'hidden_size': 400,
 'init_lr': 0.002221362095939982,
 'input_fea

07-15 06:41:13	 
07-15 06:41:13	 
07-15 06:41:13	ROUND 1
07-15 06:41:13	T	4.0
07-15 06:41:13	density_type	iaf_flow
07-15 06:41:13	ffn_num_layers	2.0
07-15 06:41:13	hidden_size	400.0
07-15 06:41:13	init_lr	0.002221362095939982
07-15 06:41:13	latent_dim	2.0
07-15 06:41:13	n_density	6.0
07-15 06:41:13	p_dropout	0.05
07-15 06:41:13	radius	2.0
0it [00:00, ?it/s]
43it [00:00, 424.75it/s]
86it [00:00, 401.75it/s]
127it [00:00, 389.43it/s]
169it [00:00, 399.31it/s]
210it [00:00, 401.39it/s]
253it [00:00, 410.15it/s]
295it [00:00, 411.60it/s]
337it [00:00, 413.17it/s]
380it [00:00, 417.79it/s]
424it [00:01, 422.97it/s]
467it [00:01, 417.20it/s]
512it [00:01, 426.50it/s]
555it [00:01, 422.98it/s]
598it [00:01, 413.67it/s]
640it [00:01, 406.36it/s]
681it [00:01, 406.19it/s]
722it [00:01, 402.38it/s]
763it [00:01, 398.78it/s]
807it [00:01, 409.08it/s]
848it [00:02, 375.85it/s]
887it [00:02, 379.69it/s]
929it [00:02, 389.24it/s]
970it [00:02, 393.58it/s]
1010it [00:02, 392.45it/s]
1050it [00:02, 37

07-15 06:41:36	Training 0th model.


07-15 06:41:36	Training 0th model.


...

07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.project_node.0.weight".
07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.project_node.0.bias".
07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.project_edge1.0.weight".
07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.project_edge1.0.bias".
07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.project_edge2.1.weight".
07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.project_edge2.1.bias".
07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.attentive_gru.edge_transform.1.weight".
07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.attentive_gru.edge_transform.1.bias".
07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.attentive_gru.gru.weight_ih".
07-15 08:56:21	Loading pretrained parameter "encoder.attfp.init_context.attentive_gru.gru.weight_hh".


07-15 08:56:23	ROUND 2 Valid ROC-AUC 0.906394898838242 +/- 0.003878843474738389
07-15 08:56:23	ROUND 2 Valid PRC-AUC 0.7856097901420828 +/- 0.005370837406183825
07-15 08:56:23	ROUND 2 Valid ACC     0.8444171779141104 +/- 0.01637749566965752
07-15 08:56:23	ROUND 2 Valid MCC     0.5901236141982402 +/- 0.05322993820149514
07-15 08:56:23	ROUND 2 Valid F1      0.6706914812200797 +/- 0.08347519892502955
07-15 08:56:23	ROUND 2 Valid ECE     0.037376779655531704 +/- 0.02982526694454454
07-15 08:56:23	ROUND 2 Valid OFR     0.04437947901255325 +/- 0.007770442350419967
07-15 08:56:23	ROUND 2 Valid OFN     0.04019986399290641 +/- 0.008810173413021894
07-15 08:56:23	ROUND 2 Valid Brier   0.10999340834271172 +/- 0.009339364763098077
07-15 08:56:23	 
07-15 08:56:23	ROUND 2 Test ROC-AUC 0.7865854832499036 +/- 0.02624453207564209
07-15 08:56:23	ROUND 2 Test PRC-AUC 0.2377324117761881 +/- 0.04238620410494743
07-15 08:56:23	ROUND 2 Test ACC     0.821505376344086 +/- 0.05455464543381438
07-15 08:56:23	ROU

07-15 08:56:23	ROUND 2 Valid ROC-AUC 0.906394898838242 +/- 0.003878843474738389
07-15 08:56:23	ROUND 2 Valid PRC-AUC 0.7856097901420828 +/- 0.005370837406183825
07-15 08:56:23	ROUND 2 Valid ACC     0.8444171779141104 +/- 0.01637749566965752
07-15 08:56:23	ROUND 2 Valid MCC     0.5901236141982402 +/- 0.05322993820149514
07-15 08:56:23	ROUND 2 Valid F1      0.6706914812200797 +/- 0.08347519892502955
07-15 08:56:23	ROUND 2 Valid ECE     0.037376779655531704 +/- 0.02982526694454454
07-15 08:56:23	ROUND 2 Valid OFR     0.04437947901255325 +/- 0.007770442350419967
07-15 08:56:23	ROUND 2 Valid OFN     0.04019986399290641 +/- 0.008810173413021894
07-15 08:56:23	ROUND 2 Valid Brier   0.10999340834271172 +/- 0.009339364763098077
07-15 08:56:23	 
07-15 08:56:23	ROUND 2 Test ROC-AUC 0.7865854832499036 +/- 0.02624453207564209
07-15 08:56:23	ROUND 2 Test PRC-AUC 0.2377324117761881 +/- 0.04238620410494743
07-15 08:56:23	ROUND 2 Test ACC     0.821505376344086 +/- 0.05455464543381438
07-15 08:56:23	ROU

100%|██████████| 2/2 [2:15:10<00:00, 4055.13s/trial, best loss: -0.9197282068153683]  


07-15 08:56:23	{"T": 4.0, "density_type": 0, "dropout": 0.05, "ffn_num_layers": 2.0, "hidden_size": 400.0, "init_lr": 0.002221362095939982, "latent_dim": 2.0, "n_density": 6.0, "radius": 2.0}
